# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [111]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score
# from scipy import stats
import xlsxwriter
from secrets import IEX_CLOUD_API_TOKEN


## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [112]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [113]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2271534200784,
 'week52high': 137.72,
 'week52low': 57.71,
 'week52change': 0.7325543852088802,
 'sharesOutstanding': 16954397618,
 'float': 0,
 'avg10Volume': 123713054,
 'avg30Volume': 120715327,
 'day200MovingAvg': 116.3,
 'day50MovingAvg': 132.18,
 'employees': 0,
 'ttmEPS': 3.35,
 'ttmDividendRate': 0.8109362284630409,
 'dividendYield': 0.006374956109336363,
 'nextDividendDate': '0',
 'exDividendDate': '2020-10-28',
 'nextEarningsDate': '0',
 'peRatio': 40.56396380583186,
 'beta': 1.171030886651185,
 'maxChangePercent': 51.50249966453937,
 'year5ChangePercent': 4.904739470127189,
 'year2ChangePercent': 2.594775667865778,
 'year1ChangePercent': 0.7796202067337666,
 'ytdChangePercent': -0.00486438815616091,
 'month6ChangePercent': 0.39047095825081873,
 'month3ChangePercent': 0.15291308908706694,
 'month1ChangePercent': 0.06317303538276345,
 'day30ChangePercent': 0.0862240879771382,
 'day5ChangePercent': -0.005017978035310843}

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [114]:
data['year1ChangePercent']

0.7796202067337666

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [115]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

# for symbol_string in symbol_strings:
#     print(symbol_string)

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [116]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings[:1]:
    batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=stats,price&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url).json()
#     print(data['AAPL']['stats'])
#     print(data['AAPL']['price'])
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
        pd.Series(
        [
            symbol,
            data[symbol]['price'],
            data[symbol]['stats']['year1ChangePercent'],
            'N/A'
        ],
        index = my_columns),
            ignore_index = True
        )
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,132.74,0.497442,N/A
1,AAL,15.14,-0.471514,N/A
2,AAP,177.91,0.122230,N/A
3,AAPL,134.74,0.768483,N/A
4,ABBV,109.04,0.271468,N/A
...,...,...,...,...
95,CINF,88.84,-0.124738,N/A
96,CL,85.93,0.224684,N/A
97,CLX,200.21,0.341055,N/A
98,CMA,62.55,-0.059530,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [117]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,78,CARR,41.47,2.507770,N/A
1,23,ALB,188.60,1.638361,N/A
2,24,ALGN,581.08,1.036568,N/A
3,31,AMD,95.46,1.003875,N/A
4,85,CDNS,138.05,0.886076,N/A
5,6,ABMD,337.40,0.852872,N/A
6,3,AAPL,134.74,0.768483,N/A
7,36,AMZN,3292.70,0.702984,N/A
8,13,ADSK,325.40,0.695959,N/A
9,63,BIO,651.50,0.647263,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [118]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter size of portfolio: ')
    
    try:
        float(portfolio_size)
    except:
        print('Please enter a valid number: ')
        portfolio_size = input('Enter size of portfolio: ')
portfolio_input()

Enter size of portfolio: 10000


In [119]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Price'])
final_dataframe

/Users/kevin/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,78,CARR,41.47,2.507770,4
1,23,ALB,188.60,1.638361,1
2,24,ALGN,581.08,1.036568,0
3,31,AMD,95.46,1.003875,2
4,85,CDNS,138.05,0.886076,1
5,6,ABMD,337.40,0.852872,0
6,3,AAPL,134.74,0.768483,1
7,36,AMZN,3292.70,0.702984,0
8,13,ADSK,325.40,0.695959,0
9,63,BIO,651.50,0.647263,0


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [148]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Raw Return',
    'One-Year % Return',
    'Six-Month Raw Return',
    'Six-Month % Return',
    'Three-Month Raw Return',
    'Three-Month % Return',
    'One-Month Raw Return',
    'One-Month % Return',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=stats,price&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['price'],
                'N/A',
                data[symbol]['stats']['year1ChangePercent'],
                'N/A',
                data[symbol]['stats']['month6ChangePercent'],
                'N/A',
                data[symbol]['stats']['month3ChangePercent'],
                'N/A',
                data[symbol]['stats']['month1ChangePercent'],
                'N/A',
                'N/A'
            ],
            index = hqm_columns),
            ignore_index = True
        )
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Raw Return,One-Year % Return,Six-Month Raw Return,Six-Month % Return,Three-Month Raw Return,Three-Month % Return,One-Month Raw Return,One-Month % Return,HQM Score
0,A,128.94,N/A,0.500583,N/A,0.443213,N/A,0.224177,N/A,0.0893764,N/A,N/A
1,AAL,15.65,N/A,-0.464577,N/A,0.273167,N/A,0.152512,N/A,-0.147756,N/A,N/A
2,AAP,173.94,N/A,0.121885,N/A,0.26169,N/A,0.0833757,N/A,0.115439,N/A,N/A
3,AAPL,134.62,N/A,0.766205,N/A,0.396141,N/A,0.151246,N/A,0.0622749,N/A,N/A
4,ABBV,107.40,N/A,0.268989,N/A,0.111301,N/A,0.249343,N/A,-0.00396139,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.86,N/A,0.0716867,N/A,0.258721,N/A,0.134125,N/A,0.0199303,N/A,N/A
501,ZBH,161.25,N/A,0.0711777,N/A,0.355383,N/A,0.0865675,N/A,0.0872796,N/A,N/A
502,ZBRA,407.02,N/A,0.648458,N/A,0.59027,N/A,0.455826,N/A,0.0826931,N/A,N/A
503,ZION,48.98,N/A,-0.0142578,N/A,0.569602,N/A,0.531019,N/A,0.135077,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [149]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        if hqm_dataframe.loc[row, f'{time_period} Raw Return'] == None:
            hqm_dataframe.loc[row, f'{time_period} Raw Return'] = 0

for row in hqm_dataframe.index:
    for time_period in time_periods:

        change_col = f'{time_period} Raw Return'
        percentile_col = f'{time_period} % Return'
#         if hqm_dataframe.loc[row, change_col] == None:
#             hqm_dataframe.loc[row, change_col] = 0
        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[f'{time_period} Raw Return'], hqm_dataframe.loc[row, f'{time_period} Raw Return'])/100
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Raw Return,One-Year % Return,Six-Month Raw Return,Six-Month % Return,Three-Month Raw Return,Three-Month % Return,One-Month Raw Return,One-Month % Return,HQM Score
0,A,128.94,N/A,0.500583,0.881188,0.443213,0.742574,0.224177,0.667327,0.0893764,0.786139,N/A
1,AAL,15.65,N/A,-0.464577,0.0138614,0.273167,0.514851,0.152512,0.524752,-0.147756,0.0039604,N/A
2,AAP,173.94,N/A,0.121885,0.520792,0.26169,0.493069,0.0833757,0.372277,0.115439,0.861386,N/A
3,AAPL,134.62,N/A,0.766205,0.966337,0.396141,0.675248,0.151246,0.516832,0.0622749,0.649505,N/A
4,ABBV,107.40,N/A,0.268989,0.70495,0.111301,0.241584,0.249343,0.69901,-0.00396139,0.281188,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.86,N/A,0.0716867,0.447525,0.258721,0.487129,0.134125,0.489109,0.0199303,0.431683,N/A
501,ZBH,161.25,N/A,0.0711777,0.443564,0.355383,0.623762,0.0865675,0.390099,0.0872796,0.776238,N/A
502,ZBRA,407.02,N/A,0.648458,0.936634,0.59027,0.877228,0.455826,0.916832,0.0826931,0.748515,N/A
503,ZION,48.98,N/A,-0.0142578,0.29505,0.569602,0.861386,0.531019,0.950495,0.135077,0.914851,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [150]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} % Return'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Raw Return,One-Year % Return,Six-Month Raw Return,Six-Month % Return,Three-Month Raw Return,Three-Month % Return,One-Month Raw Return,One-Month % Return,HQM Score
0,A,128.94,N/A,0.500583,0.881188,0.443213,0.742574,0.224177,0.667327,0.0893764,0.786139,0.769307
1,AAL,15.65,N/A,-0.464577,0.0138614,0.273167,0.514851,0.152512,0.524752,-0.147756,0.0039604,0.264356
2,AAP,173.94,N/A,0.121885,0.520792,0.26169,0.493069,0.0833757,0.372277,0.115439,0.861386,0.561881
3,AAPL,134.62,N/A,0.766205,0.966337,0.396141,0.675248,0.151246,0.516832,0.0622749,0.649505,0.70198
4,ABBV,107.40,N/A,0.268989,0.70495,0.111301,0.241584,0.249343,0.69901,-0.00396139,0.281188,0.481683
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.86,N/A,0.0716867,0.447525,0.258721,0.487129,0.134125,0.489109,0.0199303,0.431683,0.463861
501,ZBH,161.25,N/A,0.0711777,0.443564,0.355383,0.623762,0.0865675,0.390099,0.0872796,0.776238,0.558416
502,ZBRA,407.02,N/A,0.648458,0.936634,0.59027,0.877228,0.455826,0.916832,0.0826931,0.748515,0.869802
503,ZION,48.98,N/A,-0.0142578,0.29505,0.569602,0.861386,0.531019,0.950495,0.135077,0.914851,0.755446


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [151]:
hqm_dataframe.sort_values('HQM Score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace = True, drop = True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Raw Return,One-Year % Return,Six-Month Raw Return,Six-Month % Return,Three-Month Raw Return,Three-Month % Return,One-Month Raw Return,One-Month % Return,HQM Score
0,ALB,191.40,N/A,1.66642,0.99802,1.4255,0.994059,0.923065,0.99802,0.288209,0.99802,0.99703
1,FCX,32.14,N/A,1.37624,0.994059,1.48555,0.99604,0.880668,0.99604,0.276897,0.994059,0.99505
2,SIVB,445.14,N/A,0.702377,0.954455,1.05271,0.984158,0.55898,0.958416,0.200555,0.974257,0.967822
3,LB,48.73,N/A,1.65294,0.99604,1.98933,1,0.452891,0.912871,0.148367,0.928713,0.959406
4,ALGN,574.62,N/A,0.99913,0.986139,1.10111,0.988119,0.775878,0.984158,0.109556,0.851485,0.952475
5,APTV,145.93,N/A,0.580019,0.918812,0.911574,0.976238,0.49545,0.936634,0.185129,0.960396,0.94802
6,MS,78.58,N/A,0.508388,0.883168,0.585149,0.871287,0.566489,0.962376,0.182948,0.958416,0.918812
7,DE,302.91,N/A,0.72929,0.962376,0.902968,0.974257,0.264763,0.746535,0.16899,0.954455,0.909406
8,TPR,34.84,N/A,0.26107,0.687129,1.67457,0.99802,0.86206,0.994059,0.166858,0.950495,0.907426
9,EXPE,146.86,N/A,0.343207,0.80396,0.733658,0.934653,0.514795,0.946535,0.14736,0.922772,0.90198


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [152]:
portfolio_input()

Enter size of portfolio: 10000


In [153]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price'])

hqm_dataframe

/Users/kevin/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Ticker,Price,Number of Shares to Buy,One-Year Raw Return,One-Year % Return,Six-Month Raw Return,Six-Month % Return,Three-Month Raw Return,Three-Month % Return,One-Month Raw Return,One-Month % Return,HQM Score
0,ALB,191.40,1,1.66642,0.99802,1.4255,0.994059,0.923065,0.99802,0.288209,0.99802,0.99703
1,FCX,32.14,6,1.37624,0.994059,1.48555,0.99604,0.880668,0.99604,0.276897,0.994059,0.99505
2,SIVB,445.14,0,0.702377,0.954455,1.05271,0.984158,0.55898,0.958416,0.200555,0.974257,0.967822
3,LB,48.73,4,1.65294,0.99604,1.98933,1,0.452891,0.912871,0.148367,0.928713,0.959406
4,ALGN,574.62,0,0.99913,0.986139,1.10111,0.988119,0.775878,0.984158,0.109556,0.851485,0.952475
5,APTV,145.93,1,0.580019,0.918812,0.911574,0.976238,0.49545,0.936634,0.185129,0.960396,0.94802
6,MS,78.58,2,0.508388,0.883168,0.585149,0.871287,0.566489,0.962376,0.182948,0.958416,0.918812
7,DE,302.91,0,0.72929,0.962376,0.902968,0.974257,0.264763,0.746535,0.16899,0.954455,0.909406
8,TPR,34.84,5,0.26107,0.687129,1.67457,0.99802,0.86206,0.994059,0.166858,0.950495,0.907426
9,EXPE,146.86,1,0.343207,0.80396,0.733658,0.934653,0.514795,0.946535,0.14736,0.922772,0.90198


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [154]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name = 'Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [155]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [156]:
column_format = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['One-Year Raw Return', percent_template],
    'E': ['One-Year % Return', percent_template],
    'F': ['Six-Month Raw Return', percent_template],
    'G': ['Six-Month % Return', percent_template],
    'H': ['Three-Month Raw Return', percent_template],
    'I': ['Three-Month % Return', percent_template],
    'J': ['One-Month Raw Return', percent_template],
    'K': ['One-Month % Return', percent_template],
    'L': ['HQM Score', percent_template]
}

for column in column_format.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 25, column_format[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_format[column][0], column_format[column][1])


## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [157]:
writer.save()